# 481n Project Advanced Solution

In [4]:
import cayde
from cayde.well.nlpwell import NLPWell
from cayde.classifier.dtree import DecisionTreeClassifier
from cayde.classifier.ada import AdaBoosterClassifier
from cayde.classifier.svm import SVMClassifier
from bert_embedding import BertEmbedding

In [5]:
well = NLPWell("training_data.csv")
well.load_or_fetch()

well.input_cols = ["head", "body"]
well.text_cols = ["head", "body"]
well.output_col = "Stance"

In [6]:
well.df.head()

,Stance,head,body
0,1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...
1,0,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's..."
2,1,'Nasa Confirms Earth Will Experience 6 Days of...,Thousands of people have been duped by a fake ...
3,1,Banksy 'Arrested & Real Identity Revealed' Is ...,If you’ve seen a story floating around on your...
4,1,Woman detained in Lebanon is not al-Baghdadi's...,An Iraqi official denied that a woman detained...


# API Demonstration

In [7]:
# Returns a list of new columns available for usage
# This creates columns where the text is stemmed and cleaned
well.clean_text()

['head_1gram', 'head_clean', 'body_1gram', 'body_clean']

In [8]:
# This creates columns where the text is converted to bigrams/trigrams
well.createNgrams()

['head_2gram',
 'head_2gram_unique',
 'head_2gram_size',
 'head_2gram_unique_ratio',
 'head_3gram',
 'head_3gram_unique',
 'head_3gram_size',
 'head_3gram_unique_ratio',
 'head_4gram',
 'head_4gram_unique',
 'head_4gram_size',
 'head_4gram_unique_ratio',
 'body_2gram',
 'body_2gram_unique',
 'body_2gram_size',
 'body_2gram_unique_ratio',
 'body_3gram',
 'body_3gram_unique',
 'body_3gram_size',
 'body_3gram_unique_ratio',
 'body_4gram',
 'body_4gram_unique',
 'body_4gram_size',
 'body_4gram_unique_ratio']

In [9]:
# Does the word exist in each text_column?
well.createCountWordFeatures([
    'fake',
    'fraud',
    'hoax',
    'false',
    'deny', 'denies',
    'refute',
    'not',
    'despite',
    'nope',
    'doubt', 'doubts',
    'bogus',
    'debunk',
    'pranks',
    'retract'
])

['head_contains_fake',
 'head_contains_fraud',
 'head_contains_hoax',
 'head_contains_false',
 'head_contains_deny',
 'head_contains_denies',
 'head_contains_refute',
 'head_contains_not',
 'head_contains_despite',
 'head_contains_nope',
 'head_contains_doubt',
 'head_contains_doubts',
 'head_contains_bogus',
 'head_contains_debunk',
 'head_contains_pranks',
 'head_contains_retract',
 'body_contains_fake',
 'body_contains_fraud',
 'body_contains_hoax',
 'body_contains_false',
 'body_contains_deny',
 'body_contains_denies',
 'body_contains_refute',
 'body_contains_not',
 'body_contains_despite',
 'body_contains_nope',
 'body_contains_doubt',
 'body_contains_doubts',
 'body_contains_bogus',
 'body_contains_debunk',
 'body_contains_pranks',
 'body_contains_retract']

In [10]:
well.input_cols = production_cols = [
 'head_2gram_unique_ratio',
 'head_3gram_unique_ratio',
 'head_4gram_unique_ratio',
 'body_2gram_unique_ratio',
 'body_3gram_unique_ratio',
 'body_4gram_unique_ratio'
] + ['head_contains_fake',
 'head_contains_fraud',
 'head_contains_hoax',
 'head_contains_false',
 'head_contains_deny',
 'head_contains_denies',
 'head_contains_refute',
 'head_contains_not',
 'head_contains_despite',
 'head_contains_nope',
 'head_contains_doubt',
 'head_contains_doubts',
 'head_contains_bogus',
 'head_contains_debunk',
 'head_contains_pranks',
 'head_contains_retract',
 'body_contains_fake',
 'body_contains_fraud',
 'body_contains_hoax',
 'body_contains_false',
 'body_contains_deny',
 'body_contains_denies',
 'body_contains_refute',
 'body_contains_not',
 'body_contains_despite',
 'body_contains_nope',
 'body_contains_doubt',
 'body_contains_doubts',
 'body_contains_bogus',
 'body_contains_debunk',
 'body_contains_pranks',
 'body_contains_retract']

In [ ]:
well.input_cols = well.text_cols = ['head_clean']

In [ ]:
well.input_df['head_clean'][3]

In [ ]:
# df = well.createBERTEncodings()

In [ ]:
bert_vectors['head_clean'][0].shape

In [11]:
clf = DecisionTreeClassifier(well)
clf.fit()
clf.accuracy()

,f1,accuracy,recall,precision,accuracyImprovement
0,0.906689,0.849934,0.902961,0.910448,0.035857
1,0.888333,0.822045,0.878089,0.898820,0.007968
2,0.895145,0.829349,0.879006,0.911887,0.015272


In [12]:
clf = AdaBoosterClassifier(well, n_estimators=150)
clf.fit()
clf.accuracy()

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,f1,accuracy,recall,precision,accuracyImprovement
0,0.912406,0.853254,0.946546,0.880643,0.039177
1,0.900763,0.835989,0.923394,0.879216,0.021912
2,0.908151,0.846614,0.915064,0.901342,0.032537


In [13]:
clf = SVMClassifier(well)
clf.fit()
clf.accuracy()

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,f1,accuracy,recall,precision,accuracyImprovement
0,0.902668,0.828021,0.987664,0.831142,0.013944
1,0.900449,0.823373,0.990939,0.825103,0.009296
2,0.913562,0.845950,0.982372,0.853760,0.031873
